In [4]:
import os 
import numpy as np
import cv2
# import IPython
# e = IPython.embed
from pathlib import Path
import imageio
import h5py
import json
import tempfile
import lmdb
import pickle
tmp_dir = tempfile.TemporaryDirectory()

dataset_root_path = "/data/yujunqiu/code/llavavla/data/real_data/0704_2"
# camera_name = "wrist_camera"
camera_name = "camera_1"

def get_data_path(dataset_root_path):
    for root, dirs, files in os.walk(dataset_root_path):
        for file in files:
            if file.endswith(".json"):
                yield Path(root) / file


def mix_data_to_video(data_path: Path, camera_name: str):    
    with lmdb.open(str(data_path.parent / "lmdb")) as lmdb_env:
        with lmdb_env.begin() as txn:
            action = np.array(pickle.loads(txn.get("action".encode("utf-8"))))
            length = action.shape[0]
            images = []
            for i in range(length):
                render_image_key = f"observations/images/{camera_name}/{i}"
                render_image = cv2.imdecode(pickle.loads(txn.get(render_image_key.encode("utf-8"))), cv2.IMREAD_COLOR)
                images.append(np.uint8(render_image))
    return np.array(images)

iterator = get_data_path(dataset_root_path)


In [5]:
from IPython.display import Video
from loguru import logger
path = next(iterator)
logger.info(f"processing {path}")
images = mix_data_to_video(path, camera_name)
video_path = os.path.join(tmp_dir.name, f"{camera_name}_{np.random.randint(100000)}.mp4")
imageio.mimsave(video_path, np.array(images), fps=30)
Video(video_path, embed=True)

2025-07-04 10:39:32.688 | INFO     | __main__:<module>:4 - processing /data/yujunqiu/code/llavavla/data/real_data/0704_2/2025-07-04_10-27-40/log-000002-5612/_keys.json


In [3]:
import os
import lmdb
import pickle
import json

def try_deserialize_key(key_bytes):
    for try_func in [
        lambda x: pickle.loads(x),
        lambda x: json.loads(x.decode('utf-8')),
        lambda x: x.decode('utf-8'),  # 尝试普通字符串
    ]:
        try:
            return try_func(key_bytes)
        except Exception:
            continue
    return key_bytes  # fallback，原始 bytes

def print_lmdb_keys_human_readable(lmdb_path):
    print(f"\n📂 LMDB found in: {lmdb_path}")
    try:
        env = lmdb.open(lmdb_path, readonly=True, lock=False)
        with env.begin() as txn:
            cursor = txn.cursor()
            for key, _ in cursor:
                readable_key = try_deserialize_key(key)
                print(f"{readable_key}")
    except Exception as e:
        print(f"  ❌ Failed to read LMDB in {lmdb_path}: {e}")

# 示例路径
lmdb_path = str(path.parent / "lmdb")
print_lmdb_keys_human_readable(lmdb_path)



📂 LMDB found in: /data/yujunqiu/code/llavavla/data/real_data/0704/2025-07-04_09-45-40/log-000005-9027/lmdb
action
observations/ee_pose
observations/images/camera_0/0
observations/images/camera_0/1
observations/images/camera_0/10
observations/images/camera_0/100
observations/images/camera_0/101
observations/images/camera_0/102
observations/images/camera_0/103
observations/images/camera_0/104
observations/images/camera_0/105
observations/images/camera_0/106
observations/images/camera_0/107
observations/images/camera_0/108
observations/images/camera_0/109
observations/images/camera_0/11
observations/images/camera_0/110
observations/images/camera_0/111
observations/images/camera_0/112
observations/images/camera_0/113
observations/images/camera_0/114
observations/images/camera_0/115
observations/images/camera_0/116
observations/images/camera_0/117
observations/images/camera_0/118
observations/images/camera_0/119
observations/images/camera_0/12
observations/images/camera_0/120
observations/

In [5]:
with lmdb.open(lmdb_path) as lmdb_env:
    with lmdb_env.begin() as txn:
        action = np.array(pickle.loads(txn.get("action".encode("utf-8"))))
        # del_ee_grippers = np.array(pickle.loads(txn.get("del_ee_grippers".encode("utf-8"))))
action.shape

(366, 9)

In [6]:
pred_action = np.load("/data/hanxiaoshen/isaac-sim-backup/backup/isaac-sim-4.0.0-1221/src/frankapy/src/debug/pred_acts_2.npy")
pred_action.shape, action.shape

((467, 9), (467, 9))

In [7]:
((pred_action - action) ** 2).mean()

0.00019204619055910587

In [9]:
pred_action[10]

array([-1.40080398e-04, -8.38197185e-01, -1.38079745e-03, -2.59120268e+00,
       -4.22149192e-03,  1.75290557e+00,  7.64410636e-01,  3.04820275e-02,
        3.04820275e-02])

In [12]:
action[10]

array([ 1.16617652e-03, -8.38178655e-01, -1.29402850e-03, -2.58831601e+00,
       -1.93890165e-03,  1.75291687e+00,  7.62878246e-01,  4.00000000e-02,
        4.00000000e-02])

In [5]:
np.save("./debug/pred_del_ee_grippers", del_ee_grippers)